In [2]:
import numpy as np

In [3]:
import onnxruntime as ort

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

In [4]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [5]:
output_name

'output'

In [6]:
from io import BytesIO
from urllib import request

from PIL import Image

In [7]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [8]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

In [9]:
input_size = 200

In [10]:
img = download_image(url)

img = prepare_image(img, (input_size,input_size))

In [31]:
# X = np.array(img)

In [11]:
import torchvision.models as models
from torchvision import transforms

In [12]:
# ImageNet normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

preprocess = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [13]:
tensor = preprocess(img)

In [14]:
tensor[:, 0, 0]

tensor([-1.0733, -0.2150, -1.4210])

In [15]:
tensor = tensor.unsqueeze(0)

In [16]:
input_array = tensor.numpy().astype(np.float32)

In [17]:
results = session.run([output_name], {input_name: input_array})

In [19]:
predictions = results[0][0].tolist()

In [20]:
predictions

[0.09156641364097595]